In [1]:
import ipywidgets as widgets
import pandas as pd
from github import Github
import requests
from bs4 import BeautifulSoup
from copy import deepcopy as dcopy

import matplotlib.pyplot as plt
import numpy as np
import os
from io import BytesIO
from PIL import Image
import base64
import unidecode

from IPython.display import display, clear_output
from matplotlib.gridspec import GridSpec
import matplotlib.patheffects as PathEffects

In [3]:
url_listone = "https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/main/2022_2023/files/listone.csv"
listone = pd.read_csv(url_listone)

url_order = "https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/main/2022_2023/files/order.txt"
txt_order = requests.get(url_order).text
order_ids = np.asarray(txt_order.strip().replace("\n", "").replace("[", "").replace("]", "").split()).astype(int)

top_listone = listone[listone["Id"].isin(order_ids)]

url_img_giocatore = "https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/main/2022_2023/files/giocatore.png"
img_giocatore = Image.open(BytesIO(base64.b64decode(requests.get(url_img_giocatore).text)))

In [4]:
colors_role = {
    "P": "orange",
    "D": "tab:green",
    "C": "blue",
    "A": "orangered",
}

In [5]:
reset = True

check = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

html_team = widgets.HTML(value="")
image_team = widgets.Image(
    value=b"",
    format='png',
    width=500,
    height=400,
)
image_team.layout.display = "none"

def show_players(check):

    response = requests.get("https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/counter.txt")
    soup = BeautifulSoup(response.content, "html.parser")
    
    hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-3]
    
    url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/counter.txt"
    counter = int(requests.get(url_data).text)
    
    player = listone[listone["Id"]==order_ids[counter]].loc[:, ["Nome", "R", "Squadra", "Qt.A"]]
    name = player["Nome"].values[0]
    role = player["R"].values[0]
    team = player["Squadra"].values[0]
    price = player["Qt.A"].values[0]
    progression = f"{counter} / {len(order_ids)}"
    
    fig = plt.figure(constrained_layout=True, figsize=(15, 8))

    gs = GridSpec(8, 4, figure=fig)
    ax2 = fig.add_subplot(gs[0, :2])
    ax2.axis(False)
    ax3 = fig.add_subplot(gs[1:-1, :2])
    ax3.axis(False)
    ax4 = fig.add_subplot(gs[1:3, 2:])
    ax4.axis(False)
    ax5 = fig.add_subplot(gs[4:7, 2])
    ax5.axis(False)
    ax6 = fig.add_subplot(gs[4:7, 3])
    ax6.axis(False)
    
    ax2.imshow(img_giocatore)
    
    try:
        player_url = unidecode.unidecode(name).upper().replace(" ", "-").replace(".", "").replace("'", "")
        url = f"https://content.fantacalcio.it/web/campioncini/card/{player_url}.png?v=12"
        img = requests.get(url).content
        img = Image.open(BytesIO(img))
        ax3.imshow(img)
    except:        
        player_url = "NO-CAMPIONCINO"
        url = f"https://content.fantacalcio.it/web/campioncini/card/{player_url}.png?v=12"
        img = requests.get(url).content
        img = Image.open(BytesIO(img))
        ax3.imshow(img)
    
    ax4.text(0, 0, f"{name},  {role},  {team}",
             weight="extra bold", color=colors_role[role], size=30,
             path_effects=[PathEffects.withStroke(linewidth=2, foreground="black"),
                          PathEffects.withSimplePatchShadow((3, -6), alpha=0.2)],
            )
        
    min_counter = max(0, counter - 5)
    ax5.text(0, 6, f"Ultimi 5 giocatori chiamati:",
        weight="demibold", color="black", size=12,
    )
    for c_ in range(min_counter, counter):
        player_ = listone[listone["Id"]==order_ids[c_]].loc[:, ["Nome", "R", "Squadra", "Qt.A"]]
        name_ = player_["Nome"].values[0]
        role_ = player_["R"].values[0]
        team_ = player_["Squadra"].values[0]
        price_ = player_["Qt.A"].values[0]
        ax5.text(0, c_ - min_counter + 1 + 5 - (counter - min_counter), f"{name_},  {role_},  {team_}",
                 color="black", size=12,
                 path_effects=[PathEffects.withStroke(linewidth=1, foreground=colors_role[role_])],
                )
    ax5.set_ylim(0, 7)
    
    ax6.text(0, 5, f"Top rimanenti:",
        weight="demibold", color="black", size=12,
    )
    remaining_ids = order_ids[counter + 1:]
    remaining = top_listone[top_listone['Id'].isin(remaining_ids)]
    for c_, r_ in enumerate(["P", "D", "C", "A"]):
        n_total_role = len(top_listone[top_listone["R"]==r_])
        n_remaining_role = len(remaining[remaining["R"]==r_])
        ax6.text(0, 4 - c_, f"{r_}: {n_remaining_role} / {n_total_role}",
                 color=colors_role[r_], size=20, weight="extra bold",
                 path_effects=[PathEffects.withSimplePatchShadow((1.5, -1.5), alpha=0.15),]
                )
    ax6.set_ylim(0, 6)
        
    plt.show()
    
    
    url = "https://www.fantacalcio.it/news/calcio-italia/05_07_2022/serie-a-2022-23-le-probabili-formazion-per-l-asta-del-fantacalcio-oggi-giocherebbero-cosi-427972"
    content = BeautifulSoup(requests.get(url).content)

    infografica = content.find("div", {"class": "article-body"})
    sections_teams = infografica.find_all("section")

    break_after_pic = False
    for index, section in enumerate(sections_teams):

        if break_after_pic is False:
            if section["class"] != ['article-content', 'article-content-type-text']:
                continue

            team_ = section.find("h2")
            if team_ is None: continue
            team_ = team_.text

            info_text = section.text

            if team_.lower() == team.lower():
                break_after_pic = True

        else:
            if section["class"] != ['article-content', 'article-content-type-lineup']:
                continue

            info_text = info_text.split("\n")
            lines = ["<br>"*6]
            for line in info_text[3:]:
                if len(line) < 2: continue
                feature = line.split(":")[0]
                infos = line.split(":")[1].replace(name, f"<strong><a style='color: firebrick'>{name}</a></strong>")
                line = "<div style='max-width:500px; line-height: 14px;'><strong>" + feature + "</strong>: " + infos + "</div>"
                lines.append(line)

            html_team.value = "<br>".join(lines)

            img_lineup = section.find("img")["src"]
            img_lineup = requests.get(img_lineup).content
            image_team.value = img_lineup
            image_team.layout.display = ""

            break
    
    
    
def on_button_clicked(_):
    reset = True
    check.value = not check.value

button = widgets.Button(description='Aggiorna stato asta')
button.on_click(on_button_clicked)

display(widgets.HBox([button]))
widgets.interactive_output(show_players, {"check": check})

Output()

In [6]:
display(widgets.HTML(value="<br><h2>Informazioni sulla squadra</h2>"))

HTML(value='<br><h2>Informazioni sulla squadra</h2>')

In [7]:
display(widgets.HBox([html_team, image_team]))

In [8]:
check_personal = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

user_field_shown = widgets.Text(
    value='',
    placeholder='username',
    description='User:',
    disabled=False
)


user_field_hidden = widgets.Text(
    value='',
    placeholder='username',
    description='User:',
    disabled=False
)

players_bought = widgets.HTML(value="")
    

def show_personal(players_bought):
    
    if user_field_hidden.value is None or user_field_hidden.value == "":
        return
    
    list_bought_ids = [int(player_.split(",")[0]) for player_ in players_bought.split("\n") if len(player_)>1]
    
    print("\n".join([
        f"{name} -- {role} {team} ({id_})"
        for name, role, team, id_ in zip(
            listone["Nome"].values,
            listone["R"].values,
            listone["Squadra"].values,
            listone["Id"].values,
        )
        if id_ in list_bought_ids
    ]))
    
    g = Github(os.environ['GITHUB_TOKEN'])
    GITHUB_REPO = 'Asta-Fantacalcio'

    repo = g.get_user().get_repo(GITHUB_REPO)
    git_file = os.path.join("2022_2023", "user_data", f"{user_field_hidden.value}.txt")
    try:
        contents = repo.get_contents(git_file)
        repo.update_file(git_file, "committing files", players_bought, contents.sha, branch="main")
    except:
        repo.create_file(git_file, "committing files", players_bought, branch="main")

    
def login(_):
    user_field_hidden.value = user_field_shown.value.lower()
    user = user_field_hidden.value.lower()
    
    if user_field_hidden.value != "":
        login_button.layout.display = "none"
        user_field_shown.layout.display = "none"
        logout_button.layout.display = ""
        login_info.layout.display = ""
        update_personal.layout.display = ""
        add_player_button.layout.display = ""
        player_to_add.layout.display = ""
        label_price.layout.display = ""
        price_text.layout.display = ""
        remove_player_button.layout.display = ""
        login_info.value = "&emsp;"*2 + f"<strong><a style='color: darkgreen'>Logged in as {user_field_hidden.value.capitalize()} !"
        
        try:
            response = requests.get(f"https://github.com/AstaFantacalcio/Asta-Fantacalcio/blob/main/2022_2023/user_data/{user}.txt")
            soup = BeautifulSoup(response.content, "html.parser")
            hash_last_commit = soup.find("a", {"class": "d-none js-permalink-shortcut"}).get("href").split("/")[-4]
            url_data = fr"https://raw.githubusercontent.com/AstaFantacalcio/Asta-Fantacalcio/{hash_last_commit}/2022_2023/user_data/{user}.txt"
            players_bought.value = requests.get(url_data).text
        except:
            players_bought.value = ""
            
def logout(_):
    
    login_button.layout.display = ""
    user_field_shown.layout.display = ""
    logout_button.layout.display = "none"
    login_info.layout.display = "none"
    update_personal.layout.display = "none"
    add_player_button.layout.display = "none"
    player_to_add.layout.display = "none"
    label_price.layout.display = "none"
    price_text.layout.display = "none"
    remove_player_button.layout.display = "none"
    player_to_remove.layout.display = "none"
    confirm_remove_player_button.layout.display = "none"
    


login_button = widgets.Button(description='Login')
login_button.on_click(login)

logout_button = widgets.Button(description='Logout')
logout_button.layout.display = "none"
logout_button.on_click(logout)

login_info = widgets.HTML(value="")
login_info.layout.display = "none"

update_personal = widgets.Button(description='Aggiorna Info Utente')
update_personal.layout.display = "none"
update_personal.on_click(show_personal)

def add_player(_):
    
    added_player = player_to_add.value
    if added_player == "": return
    id_ = added_player.split("(")[-1][:-1]
    
    price = price_text.value  
    
    player_to_add.value = ""
    price_text.value = 0
    
    players_bought.value = players_bought.value + f"\n{id_}, {price}"

add_player_button = widgets.Button(description='Aggiungi giocatore')
add_player_button.on_click(add_player)
add_player_button.layout.display = "none"

player_to_add = widgets.Combobox(
    placeholder='Scegli giocatore',
    options=[
        f"{name} -- {role} {team} ({id_})"
        for name, role, team, id_ in zip(
            listone["Nome"].values,
            listone["R"].values,
            listone["Squadra"].values,
            listone["Id"].values,
        )
    ],
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width='200px'),
)
player_to_add.layout.display = "none"

label_price = widgets.HTML(value="&emsp;&emsp;Prezzo d'acquisto: ", layout=widgets.Layout(width='130px'))
label_price.layout.display = "none"

price_text = widgets.IntText(visible=False, layout=widgets.Layout(width='60px'))
price_text.layout.display = "none"

def remove_player(_):
    
    if players_bought.value == "": return
    
    list_bought_ids = [int(player_.split(",")[0]) for player_ in players_bought.value.split("\n") if len(player_)>1]
    
    player_to_remove.options = [
        f"{name} -- {role} {team} ({id_})"
        for name, role, team, id_ in zip(
            listone["Nome"].values,
            listone["R"].values,
            listone["Squadra"].values,
            listone["Id"].values,
        )
        if id_ in list_bought_ids
    ]
    
    player_to_remove.value = ""
    player_to_remove.layout.display = ""
    confirm_remove_player_button.layout.display = ""
    cancel_remove_player_button.layout.display = ""

remove_player_button = widgets.Button(description='Rimuovi giocatore')
remove_player_button.on_click(remove_player)
remove_player_button.layout.display = "none"

player_to_remove = widgets.Combobox(
    placeholder='Scegli giocatore',
    options=[""],
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width='200px'),
)
player_to_remove.layout.display = "none"

def confirm_remove_player(_):
    
    removed_player = player_to_remove.value
    if removed_player == "": return
    remove_id = removed_player.split("(")[-1][:-1]
    
    player_to_remove.layout.display = "none"
    confirm_remove_player_button.layout.display = "none"
    cancel_remove_player_button.layout.display = "none"
    
    remaining_players = [line for line in players_bought.value.split("\n") if not line.startswith(remove_id)]
    if len(remaining_players) == 0:
        players_bought.value = ""
    else:
        players_bought.value = "\n".join(remaining_players)
    
def cancel_remove_payer(_):
    
    player_to_remove.layout.display = "none"
    confirm_remove_player_button.layout.display = "none"
    cancel_remove_player_button.layout.display = "none"

confirm_remove_player_button = widgets.Button(description='Conferma')
confirm_remove_player_button.on_click(confirm_remove_player)
confirm_remove_player_button.layout.display = "none"

cancel_remove_player_button = widgets.Button(description='Annulla')
cancel_remove_player_button.on_click(cancel_remove_payer)
cancel_remove_player_button.layout.display = "none"

login_box = widgets.HBox([login_button, user_field_shown])
logout_box = widgets.HBox([logout_button, login_info])
add_player_box = widgets.HBox([add_player_button, player_to_add, label_price, price_text])
remove_player_box = widgets.HBox([remove_player_button, player_to_remove, confirm_remove_player_button, cancel_remove_player_button])
grid_widgets = widgets.VBox([login_box, logout_box, add_player_box, remove_player_box])

display(grid_widgets)
widgets.interactive_output(show_personal, {"players_bought": players_bought})

Output()